In [ ]:
import lyricsgenius
import os
from dotenv import load_dotenv
load_dotenv()

## Fetch Data

In [ ]:
os.chdir("./data/raw/")

In [ ]:
offset = 116
clientID = os.getenv("clientID")
clientAccessToken = os.getenv("clientAccessToken")
clientSecret = os.getenv("clientSecret")

In [ ]:
def fetchAndSave(artist, genius):
    print("search for " + artist)
    artist = genius.search_artist(artist, max_songs=500, per_page= 50, sort="popularity", get_full_info= False)
    print("saving lyrics")
    artist.save_lyrics()


In [ ]:
with open("../../topRappers.txt", "r") as fp:
    genius = lyricsgenius.Genius(client_access_token = clientAccessToken, verbose=False, timeout = 15)
    genius.excluded_terms = ["(Remix)", "(Live)"]
    count = 0
    for i, artist in enumerate(fp):
        # print(artist)
        # print (i)
        if i >=  offset :
            try:
                fetchAndSave(artist, genius)
            except TimeoutError:
                continue

## Clean Data

In [3]:
import json
files = os.listdir("./data/raw/")

In [ ]:
ignorelist = ["Lyrics will be available" , "Cover Art", "Lyrics for this song have yet to be released", "Lyrics will be put on this page once the song is released",
"Chance the Rapper Logo", "Someone add the lyrics", ]

In [ ]:
for filename in files:
    # temp = ""
    # filename = "Lyrics_Dax.json"
    with open(f'./data/raw/{filename}') as f:
        data = json.load(f)
        song_obj = {}
        artist = data["name"]
        song_obj["artist"] = artist

        song_obj["songs"] = []
        for song in data['songs']:
            lyrics = song['lyrics']
            if "My Krazy Life Tour" in song["title"]:
                continue
            if any(ignore in lyrics for ignore in ignorelist):
                continue
            if len(lyrics)<20:
                if "Soon" in lyrics:
                    continue
            # lyrics.decode("utf-8")
            # print(lyrics)
            # temp = lyrics
            song_obj["songs"].append(song['lyrics'])
        with open(f'./data/cleaned/{artist}.json', 'w', encoding='utf-8') as outfile:
            json.dump(song_obj, outfile)

In [5]:
def encodeString(string, tokenizer, maxSize):
    encoding = tokenizer.encode(string)
    # print("*******************************")
    # print(string)
    if len(encoding) > maxSize:
        splits = string.split("\n\n")
        if len(splits) ==1:
            # return [tokenizer.encode(string, max_length=1023, truncation='only_first')]
            return[string]
        
        if len(splits) == 2:
            res = []
            # res.append(tokenizer.encode(splits[0], max_length=1023, truncation='only_first'))
            # res.append(tokenizer.encode(splits[1], max_length=1023, truncation='only_first'))
            res.append(splits[0])
            res.append(splits[1])
            return res
            # print (splits)
        # print(len(splits))
        # for split in splits:
        #     print(len(split))
        middle = math.floor((len(splits) - 1) / 2)
        # print(middle)
        first = ""
        second = ""
        for i in range (middle):
            if i<middle-1:
                first += splits[i] + "\n\n"
            else:
                first += splits[i]

        for i in range(middle, len(splits)):
            if i< len(splits)-1:
                second += splits[i] + "\n\n"
            else:
                second+= splits[i]
        res = []

        res1 = encodeString(first, tokenizer, maxSize)
        res2 = encodeString(second, tokenizer, maxSize)
        for enc in res1:
            res.append(enc)
        for enc in res2:
            res.append(enc)
        return res
    else:
        return [string]

In [6]:
import re
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
import math

files = os.listdir("./data/cleaned/")
chorusCount = 0
verseCount = 0
songCount = 0
# chorusRE = re.compile('\[.*chorus.*\]|(.*chorus.*)|Chorus.*:|{.*chorus.*}|\[.*interlude\]|\[Chorus\]|\[Intro\]|\[hook.*\]|\[outro.*\]|\[...*\]', re.IGNORECASE)
chorusRE = re.compile('(.*chorus.*)|Chorus.*:|{.*chorus.*}|\[...*\]', re.IGNORECASE)

verseRE = re.compile("\[.*verse.*\]|(.*verse.*)|verse*:|{.*verse.*}", re.IGNORECASE)
# print("reset")
longerCount = 0
for filename in files:
    with open(f'./data/cleaned/{filename}', encoding = "utf-8") as f:
        data = json.load(f)
        artist = data["artist"]
        songs = data["songs"]
        encObj = {}
        encObj["artist"] = artist
        encObj["encodings"] = []
        for song in songs:
            # if "Southside, Southside on the, Southside on the," in song:
            #     print (data["artist"])
            # print(song[:100])
            encodings = encodeString(song, tokenizer, 1023)
            for enc in encodings:
                obj = {}
                obj["song"] = enc
                encObj["encodings"].append(obj)
            # print(len(encodings))
            # for enc in encodings:
            #     print (len)

        with open(f'./data/encodings/{artist}.json', 'w', encoding='utf-8') as outfile:
            json.dump(encObj, outfile)

            # # print(song)
            # choir = chorusRE.findall(song)
            # verses = verseRE.findall(song)

            # if len(choir) > 0 and len(verses) >0:
            #     chorusCount += 1
            #     verseCount +=1
            #     # chorus += len(choir)
            # elif len(verses) >0:
            #     verseCount +=1
            # elif len(choir)>0:
            #     chorusCount += 1
            # else:
            #     if len(song) < 500:
            #         print(song)
            #     # print(len(song))
            #     break
            # songCount +=1
        
# print(chorusCount)
# print(songCount)
# print(verseCount)

print(longerCount)

0


In [ ]:
test =      "[Intro]\nSouthside, Southside on the, Southside on the, hey\nMetro Boomin want some more, nigga\n\n[Verse]\nGoyard duffle, yeah yeah\nGot that Draco in it, yeah yeah\nWhip a chicken, yeah yeah\nTryna break the pot, yeah yeah\nAnd I got that Glock, yeah yeah\nI got thirty shots, yeah yeah\nGot your baby mama, yeah yeah\nShe bent over nigga, yeah yeah\nYoung Savage trappin', yeah yeah\nAnd I'm fuckin' rappin', yeah yeah\nI make lots of money, yeah yeah\nAnd I throw it, throw it, yeah yeah\nAyy lil bitch you tryna blow it, yeah\nWoah, chopper like a lawnmower, yeah\nWoah, got that motherfuckin' dagger on me\nWoah, stab a nigga and his lil homie, woah\nAnd I'm ballin' like Nowitzki, woah\nI get head like Lewinsky\nPolice tryna fingerprint me, woah\nI'm runnin' out that motherfucker, nigga, woah\nGet money like a motherfucker, nigga\nBags in, nigga gas in\nCash out, nigga cash in\nI'm robbin', ski maskin'\nYou niggas know that I'm 'bout action\nFishtail with the traction\nFuck a fistfight, nigga I was blasting\nAir that motherfucker out, we was smashin'\nGot them VVS's on me, nigga glass\nFuck school nigga, always skipped class\nFuck the other side, throwin' red rags\nSlaughter Gang, PDE gon' do you bad nigga\nRunnin' through this cash, through this cash nigga\nAnd my bitch bad, bitch bad nigga\nYeah, and I got a bag, got a bag nigga\nWhy these niggas mad, why they sad nigga\n'Cause I'm poppin' ho, yeah I'm poppin' ho\nGot your main bitch, pussy poppin' ho\nWe don't want that pussy, we just want that toppy, ho\nAnd I don't want it if the shit ain't sloppy ho\nYeah, gang gang, nigga gang gang\nAyy, niggas snitching in the chain gang, ayy\nThey must not know they go in the chain gang, ayy\nStab 'em up, nigga stab 'em up\nI'm in a Bentley truck, you in a Acura\nYou niggas actin' like I ain't got racks or somethin'\nYou niggas actin' like I ain't in the trap or somethin'\nFuck you niggas thought, Savage was just rappin' huh\nHey, your main bitch, she gon' back it up\n21, that two-two-three gon' make you back it up\n21, that show money I just stack it up\n21, I'm real gangster, you just acting tough, woah\nNiggas tryna put the feds on me, woah\nMy young niggas droppin' shells on 'em, woah\nI know they scared now, scared now\nNiggas broke, they can't even pay they bail now\nAyy, I get a brick and I bust it down, nigga\nPut it in the pyrex and drown it, nigga\nWater whip the dope, water whip it\nHo, water whip your ho, water whip 'em, yeah\nThem VVS's got her hypnotized, yeah\nI used to walk around with two nines, woah\nThen I hit a nigga for two nines, 21\nThat's a half a brick lil nigga, 21\nYou a fuckin' shrimp lil nigga, 21\nI got somethin' for that lip lil nigga, 21\nHave you with a limp lil nigga, 21\nCut you up, no temp lil nigga, yeah\nI get money 'cause I'm rich nigga, yeah\nI might fuck all on your bitch nigga, yeah\nGot a drank in in the vip nigga, yeah\nShe want a gangster, not a wimp nigga, yeah\nBurberry fur on me, shit, woah\nThirty, Steph Curry on me, shit, woah\nEating chicken curry in this bitch, woah\nGood food, nigga good food, 21\nI'm a gangster, you a good dude, 21\nI won't text her 'til she send nudes, 21\nShe call my phone when the rent due, 21\nI ain't got a dime for your stankin' ass, 21\nWash your behind with your stankin' ass, bitch\nAnd your nigga broke, down bad, bitch\nAll my niggas havin' fuckin' bags, bitch\nRich niggas, rich niggas, rich, 21\nHit nigga, hit nigga, hit, 21\nThat stick nigga, stick nigga, stick, 21\nBricks nigga, licks nigga, hits, 21\nSlaughter Gang, PDE the shit, 21\nGot the rap game on lock fast, 21\nGot the block game on lock fast, 21\nI got that Glock aim on lock fast, 21\nI drop cop and send shots fast, 21\nYou only gangster when the gangsters gone, 21\nYou just a pussy with a camera phone, 21\nYou just a bitch with an Instagram, 21\nNo trigger finger, I got a trigger hand, 21\nAll of 'em like to shoot nigga, 21\nAll of 'em like boot nigga, 21\nAll of 'em in your boot nigga, 21\nFinger holes, nigga finger holes, 21\nYou just a pussy with some finger rolls, 21\nI can't respect a nigga with cornrows, 21\nBroke nigga with your brother's clothes, 21\nBroke nigga fuckin' your brother's hoes, 21\nGet your own nigga, get your own, 21\nYou ain't grown, nigga you ain't grown, 21\nYour money brown and my money long, 21\nMy blunt green and my gas strong, 21\nYour bitch bad and I got her number, 21\nAll her friends wanna throw a slumber, 21\nI might fuck all on that bitch mama, 21\nI might take that lil bitch out that Honda, 21\nPut that lil bitch in a Benz, yeah, 21\nI wish that bitch had a twin, yeah, 21\nShe want all my friends, yeah, 21\nMan that bitch got on some Timbs, yeah, 21\nBut I put that bitch in Gucci Gucci, 21\nMan I fuck that bitch like she Karruche, 21\nI might lick that, do the oochie coochie, 21\nGot a brown one like a nigga Boosie, 21\nNigga catch me all up in Bruce's, 21\nAnd you know I pull up with that uzi, 21\nI'ma shoot a nigga, make the news and, 21\nNiggas umm, niggas niggas losin', 21\nYeah yeah, nigga yeah yeah, 21\nServe a nigga with no scale scale, 21\nCome and help me wrap this bale, bale, 21\nFuck 12, no seatbelt, 21\nFuck 12, I don't need help, 21\n12 gauge, nigga that's my help, 21"


In [ ]:
len(tokenizer.tokenize(test))

In [ ]:
len(encodeString(test, tokenizer, 1023)[1])

In [7]:
files = os.listdir("./data/encodings/")
source = ["./data/encodings/" + filename for filename in files ]

In [9]:
from datasets import load_dataset
dataset = load_dataset('json', data_files=source, field = 'encodings' )

Using custom data configuration default
121 tables [00:02, 49.03 tables/s]Dataset json downloaded and prepared to /home/dmh/.cache/huggingface/datasets/json/default-0dc0d5f9fcdf97b2/0.0.0/70d89ed4db1394f028c651589fcab6d6b28dddcabbe39d3b21b4d41f9a708514. Subsequent calls will reuse this data.


In [10]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['song'],
        num_rows: 36452
    })
})


In [17]:
dataset["train"][20]["song"]

"[Verse 2: Drake, Lil Wayne, Both & All]\nDrank in my cup holder, hope this shit don't spill\nPull up in the new edition and that's word to Johnny Gill\nHow I come up with this shit and all these verses that I kill?\nI have no imagination, everything I do for real\nBitch I'm camouflaged, down put your camera phone down\nIf she got a ass and the girl a fan, it's going down\nI'ma fuck you like I've been waiting a century for it\nGive the pussy up and I'll trade you the memory for it\nIn the bedroom forever, that's what her roommate will tell you\nMan I just hear this shit and think about what Tunechi will tell you\nHe might call up Patricia, she 'bout to call up Melissa\nTell 'em come to the cribbo and do them both, double dribble\nI'm colder than a hospital, she love the dick that I give her\nHit her from the front, back, side, twist her like cigarillos\nI put the gun to the pillow, I don't want blood on my clothes\nGotta keep that Trukfit fresh, shoutout to all of my hoes\n(Tunechi) Th